The dataset for this BINARY CLASSIFIER to be used is [bc-dataset.csv].
There are 2 implementations here:
1. Bag of words approach.
2. **Word vectors (can be pre-trained word embeddings).**

The dataset split is 60-40.
Evaluation metrics to be used in this are:
1. Precision.
2. Recall.
3. F-Measure.

Loads the dataset.

https://towardsdatascience.com/pre-trained-word-embedding-for-text-classification-end2end-approach-5fbf5cd8aead

In [61]:
from ftfy import fix_encoding
import pandas as pd
import numpy as np
import re
import csv
import string
import nltk as nlp
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

nlp.download("stopwords")
nlp.download('punkt')
nlp.download('wordnet')

#For GloVe
GLOVE_FILENAME = 'glove.42B.300d.txt'
STOP_WORDS = stopwords.words("english")

def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_punct(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)

def fix_encode(x):
    return fix_encoding(x)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [13]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

df = pd.read_csv(
    "datasets/bc-dataset.csv",
    encoding="latin1",
    sep=",",
    quoting=csv.QUOTE_ALL
)

data = pd.concat([df.gender, df['gender:confidence'], df.description], axis=1)

#drop null rows
print("Data Shape: " + str(data.shape))

Data Shape: (20050, 3)


In [14]:
# print(str(data.shape))

data.dropna(subset=['description', 'gender'], inplace=True)
data = data.reset_index(drop=True)

# print(str(data.shape))

data.drop(data[data['gender:confidence'] < 0.80].index, inplace=True)
data.drop('gender:confidence', axis=1, inplace=True)

data.description = data.description.apply(lambda x: fix_encode(x))
# print(str(data.shape))

In [15]:
data['description']=data['description'].apply(lambda x : remove_URL(x))
data['description']=data['description'].apply(lambda x : remove_html(x))
data['description']=data['description'].apply(lambda x : remove_emoji(x))
data['description']=data['description'].apply(lambda x : remove_punct(x))

In [21]:
print("ORIGINAL: ", data.description.iloc[213])

ORIGINAL:  waiting for Gandalf to come to my house and invite me on an adventure from which I will not expect to return


In [22]:
lemma = nlp.WordNetLemmatizer()
description_list = []

for each in data.description:
    each = re.sub("[^a-zA-Z]"," ", str(each))                                        # regex to clean unnecesarry chars
    each = each.lower()                                                              # lowercase all
    each = nlp.word_tokenize(each)                                                   # split all by tokenizing
    each = [word for word in each if not word in set(stopwords.words("english"))]    # delete stop words from your array
    each = [lemma.lemmatize(word) for word in each]                                  # lemmatize "memories" -> "memory"
    each = " ".join(each)                                                            # make them one string again
    # each = correct_spellings(each)                                                   # correct the spelling (?)
    description_list.append(each)                                                         # put them into big array

# data['description'] = data['description'].apply(tweet_cleaning)

In [25]:
print("ORIGINAL: ", data.description.iloc[213])
print("PREPROCESSED: ", description_list[213])

ORIGINAL:  waiting for Gandalf to come to my house and invite me on an adventure from which I will not expect to return
PREPROCESSED:  waiting gandalf come house invite adventure expect return


In [26]:
get_female = data["gender"] == "female"
get_male = data["gender"] == "male"
get_brand = data["gender"] == "brand"

female_rows = data[get_female]
male_rows = data[get_male]
brand_rows = data[get_brand]

print("total female tweets: ", female_rows.description.count())
print("total male tweets:   ", male_rows.description.count())
print("total brand tweets:  ", brand_rows.description.count())

total female tweets:  4620
total male tweets:    4155
total brand tweets:   3011


In [27]:
female_rows.gender = 0     # female
male_rows.gender = 1       # male
brand_rows.gender = 2      # brand) 

frames = [female_rows, male_rows, brand_rows]
data = pd.concat(frames, ignore_index=True)

C:\Users\LENOVO\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [43]:
from sklearn.model_selection import train_test_split

y = data.gender.values

X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.4)

In [60]:
from tensorflow.python.keras.preprocessing import sequence
from tensorflow.python.keras.preprocessing import text

TOP_K = 20000 #number of features
MAX_SEQ_LEN = 50 #length of text sequences

class BC_Tokenizer():
    def __init__(self, train_text):
        self.train_text = train_text
        self.tokenizer = Tokenizer(num_words=TOP_K)
    
    def train_tokenize(self):
        max_len = len(max(self.train_text, key=len))
        self.max_len = min(max_len, MAX_SEQ_LEN)

        # vocabulary
        self.tokenizer.fit_on_texts(self.train_text)
    
    def vectorize_input(self, texts):
        #vectorize training
        texts = self.tokenizer.texts_to_sequences(texts)
        texts = sequence.pad_sequences(texts, maxlen=self.max_len, truncating='post', padding='post')
        return texts
    
tokenizer = BC_Tokenizer(train_text=X_train.description)

#fit
tokenizer.train_tokenize()
tokenized_train = tokenizer.vectorize_input(X_train['description'])
tokenized_test = tokenizer.vectorize_input(X_test['description'])

In [37]:
import tqdm

EMBEDDING_VECTOR_LENGTH = 50 #<=200

def construct_embedding_matrix(glove_file, word_index):
    embedding_dict={}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values=line.split()
            # get the word
            word=values[0]
            if word in word_index.keys():
                # get the vector
                vector = np.asarray(values[1:], 'float32')
                embedding_dict[word] = vector #OOVs mapped to 0 vectors
    
    num_words = len(word_index)+1
    embedding_matrix = np.zeros((num_words, EMBEDDING_VECTOR_LENGTH))

    for word, i in tqdm.tqdm(word_index.items()):
        if i < num_words:
            vect=embedding_dict.get(word, [])
            if len(vect)>0:
                embedding_matrix[i] = vect[:EMBEDDING_VECTOR_LENGTH]
    return embedding_matrix

embedding_matrix = construct_embedding_matrix(GLOVE_FILENAME, tokenizer.tokenizer.word_index)

100%|██████████| 19491/19491 [00:00<00:00, 207886.55it/s]


In [45]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [51]:
print(embedding_matrix.shape[0])
print(len(tokenizer.tokenizer.word_index)+1)

19492
19440
52


In [57]:
from keras.layers import Embedding,LSTM,Dense,Dropout
from keras.initializers import Constant
from keras.optimizers import Adam

model = Sequential()

embedding = Embedding(len(tokenizer.tokenizer.word_index)+53,#unique tokens
                        EMBEDDING_VECTOR_LENGTH, #no. of features
                        embeddings_initializer=Constant(embedding_matrix), #initialize
                        input_length=MAX_SEQ_LEN,
                        trainable=False
                        )

model.add(embedding)
model.add(Dropout(0.2))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.5))
model.add(Dense(1, activation='sigmoid'))

#compile
optimizer = Adam(clipvalue=0.5) #clip value to prevent the gradient exploding (?)

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['acc', f1_m, precision_m,recall_m])

history = model.fit(tokenized_train, y_train, 
                    batch_size = 32,
                    epochs=20,
                    validation_data=(tokenized_test,y_test),
                    verbose=2)


Epoch 1/20
221/221 - 30s - loss: 0.3047 - acc: 0.3479 - f1_m: 0.7493 - precision_m: 0.6133 - recall_m: 0.9965 - val_loss: -1.8030e-02 - val_acc: 0.3591 - val_f1_m: 0.7578 - val_precision_m: 0.6143 - val_recall_m: 1.0000
Epoch 2/20
221/221 - 16s - loss: -2.3550e-01 - acc: 0.3478 - f1_m: 0.7487 - precision_m: 0.6040 - recall_m: 0.9965 - val_loss: -1.2194e+00 - val_acc: 0.3591 - val_f1_m: 0.7578 - val_precision_m: 0.6143 - val_recall_m: 1.0000
Epoch 3/20
221/221 - 17s - loss: -9.8919e-01 - acc: 0.3490 - f1_m: 0.7501 - precision_m: 0.6046 - recall_m: 0.9985 - val_loss: -2.0500e+00 - val_acc: 0.3591 - val_f1_m: 0.7578 - val_precision_m: 0.6143 - val_recall_m: 1.0000
Epoch 4/20
221/221 - 18s - loss: -1.6633e+00 - acc: 0.3534 - f1_m: 0.7517 - precision_m: 0.6168 - recall_m: 0.9797 - val_loss: -2.9738e+00 - val_acc: 0.3644 - val_f1_m: 0.7603 - val_precision_m: 0.6273 - val_recall_m: 0.9763
Epoch 5/20
221/221 - 15s - loss: -2.4219e+00 - acc: 0.3473 - f1_m: 0.7481 - precision_m: 0.6049 - recall_

In [58]:
loss, accuracy, f1_score, precision, recall = model.evaluate(tokenized_test, y_test, verbose=0)

In [59]:
print("F1 Score : ", f1_score)
print("Precision : ", precision)
print("Recall : ", recall)

# print(f'F1 Score: {"#.5f"%f1_score}')
# print(f'Precision: {"#.5f"%precision}')
# print(f'Recall: {"#.5f"%recall}')

F1 Score :  0.7572255730628967
Precision :  0.6140822172164917
Recall :  0.9986042380332947
